# 第2章 型を信じろ！

## 2.1 明示的な型宣言

In [1]:
:t 'a'

'a' :: Char

In [2]:
:t True

True :: Bool

In [3]:
:t "HELLO!"

"HELLO!" :: [Char]

In [4]:
:t (True, 'a')

(True, 'a') :: (Bool, Char)

In [5]:
:t 4 == 5

4 == 5 :: Bool

GHCiでは型宣言まで明示的に書く方法が少しめんどくさいらしい。
Jupyterは関係ない。

Jupyterはいいぞ。

In [6]:
removeNonUppercase :: [Char] -> [Char]
removeNonUppercase st = [c | c <- st, c `elem` ['A'..'Z']]

Line 1: Use String
Found:
[Char] -> [Char] removeNonUppercase st
Why not:
String -> String removeNonUppercase st

`String`は`[Char]`と同じ。

In [7]:
addThree :: Int -> Int -> Int -> Int
addThree x y z = x + y + z

## 2.2 一般的なHaskellの型

`Int` は整数型。有界。

`Integer` も整数型。**はんぱなく**大きい数。

In [8]:
factorial :: Integer -> Integer
factorial n = product [1..n]

In [9]:
factorial 50

30414093201713378043612608166064768844377641568960512000000000000

`Float` は単精度浮動小数点数。

In [10]:
circumference :: Float -> Float
circumference r = 2 * pi * r

In [11]:
circumference 4.0

25.132742

`Double` は倍精度浮動小数点数。

In [12]:
circumference' :: Double -> Double
circumference' r = 2 * pi * r

In [13]:
circumference' 4.0

25.132741228718345

`Bool` は真理値型。 `True` or `False`

`Char` はUnicode文字。シングルクオートで括ろう。

タプルも型。要素をいくつでも含められ、理論的に無限種類のタプルが作れる。（実際の処理系では64個の要素が上限らしい。）

## 2.3 型変数

リストを処理する関数はリストの中身の型を知らなくてもいいのでは？

In [14]:
:t head

head :: forall a. [a] -> a

`a` を**型変数**と呼ぶ。別に一文字じゃなくてもおk

ジェネリクスっぽい。C++のテンプレートとか。

`head` のような関数を**多相的関数**と呼ぶ。

In [15]:
:t fst

fst :: forall a b. (a, b) -> a

`fst` は２つの要素を持つタプルを引数に取り前の要素の型の値を返すことがわかる。
後ろの要素の型は関係ない。

## 2.4 型クラス初級講座

**型クラス**とは？

何らかの振る舞いを定義するもの。

型クラスの**インスタンス**である型は、その型クラスが記述する振る舞いを実装する。

ある型クラスに属する関数を**メソッド**と呼ぶ。

ある型を型クラスのインスタンスにしようと考えたときには、それらの関数がその型ではどういう意味を成すのかを定義する。

`Rust`の`Trait`と同じ。オブジェクト指向言語の**クラスとは異なる。**

実際の作り方は第7章。

In [16]:
:t (==)

(==) :: forall a. Eq a => a -> a -> Bool

ここの`Eq a =>`の部分は**型クラス制約**と呼ぶ。

### Eq型クラス

Haskellのすべての標準型（I/O型と関数を除く）はEqのインスタンス。

In [17]:
5 == 5

True

In [18]:
5 /= 5

False

In [19]:
'a' == 'a'

True

In [20]:
"Ho Ho" == "Ho Ho"

True

In [21]:
3.432 == 3.432

True

`==`と`\=`が定義されている。

### Ord型クラス

順序比較ができる。全順序。

`>`、`<`、`>=`, `<=`, `compare`をサポートしてる。

`compare`は`Ordering`型を返す。`Ordering`は`LT`、`GT`、`EQ`のいずれかの値を取る。

In [22]:
"Abrakadabra" < "Zebra"

True

In [23]:
"Abrakadabra" `compare` "Zebra"

LT

In [24]:
5 >= 2

True

In [25]:
5 `compare` 3

GT

In [26]:
'b' > 'a'

True

### Show型クラス

文字列として表現できる。つまりプリントできる。

今まで出てきた**型**は関数を除いてすべて`Show`型クラスのインスタンス。

`show`関数がよく使われる。

In [27]:
show 3

"3"

In [28]:
show 5.344

"5.344"

In [29]:
show True

"True"

### Read型クラス

In [30]:
read "True" || False

True

In [3]:
read "True" :: Bool

True

In [4]:
read "4 < 6" :: Bool

Prelude.read: no parse

In [31]:
read "8.2" + 3.8

12.0

In [32]:
read "5" - 2

3

In [33]:
read "[1,2,3,4]" ++ [3]

[1,2,3,4,3]

In [34]:
read "4"

Prelude.read: no parse

`read "4"`はなんの型で返せばいいのか分からない。かっこよく言うと型推論できない。

In [35]:
:t read

read :: forall a. Read a => String -> a

こんなときは型注釈を使う。

In [36]:
read "5" :: Int

5

In [37]:
read "5" :: Float

5.0

In [38]:
(read "5" :: Float) * 4

20.0

In [39]:
read "[1,2,3,4]" :: [Int]

[1,2,3,4]

In [40]:
read "(3, 'a')" :: (Int, Char)

(3,'a')

In [41]:
[read "True", False, True, False]

[True,False,True,False]

リストはすべて同じ型の要素を持つので型を推論できる。

### Enum型クラス

`Enum`のインスタンスは要素の値を列挙できる型。
主な利点は、その値をレンジの中で使えること。

後者関数`succ`と前者関数`pred`も定義されている。

主なインスタンスとしては`()`、`Bool`、`Char`、`Ordering`、`Int`、`Integer`、`Float`、`Double`などがある。

In [42]:
['a'..'e']

"abcde"

In [43]:
[LT .. GT]

[LT,EQ,GT]

In [44]:
[3 .. 5]

[3,4,5]

In [45]:
succ 'B'

'C'

### Bounded型クラス

`Bounded`のインスタンスは上限と下限を持ち、それぞれ`minBound`と`maxBound`関数で調べられる。

In [46]:
minBound :: Int

The type signature for `minBound' lacks an accompanying binding
  (The type signature must be given where `minBound' is declared)

In [47]:
minBound :: Bool

The type signature for `minBound' lacks an accompanying binding
  (The type signature must be given where `minBound' is declared)

In [48]:
minBound

()

### Num型クラス

数値。

In [49]:
:t 20

20 :: forall a. Num a => a

In [50]:
:t 20.0

20.0 :: forall a. Fractional a => a

あらゆる数も**多相定数**として表現されていて、Num型クラスの任意のインスタンスとして振る舞うことができる。

In [51]:
20 :: Int

20

In [52]:
20 :: Integer

20

In [53]:
20 :: Float

20.0

In [54]:
20 :: Double

20.0

演算子`*`の型も調べることができる。

In [55]:
:t (*)

(*) :: forall a. Num a => a -> a -> a

注意としてある型を`Num`のインスタンスにするには、その型が既に`Show`と`Eq`のインスタンスになっている必要がある。

これも`Rust`の`Trait`と一緒。

### Floating型クラス

浮動小数点数。`Float`と`Double`がある。

### Integral型クラス

整数。`Int`と`Integer`がある。

`Integral`から他の`Num`に変換するには`fromIntegral`関数を使う。

In [56]:
:t fromIntegral

fromIntegral :: forall a b. (Integral a, Num b) => a -> b

In [57]:
length [1,2,3,4] + 3.2

No instance for (Fractional Int) arising from the literal `3.2'
In the second argument of `(+)', namely `3.2'
In the expression: length [1, 2, 3, 4] + 3.2
In an equation for `it': it = length [1, 2, 3, 4] + 3.2

In [58]:
fromIntegral (length [1,2,3,4]) + 3.2

7.2

### 最後に

型クラスは抽象的なインターフェースとして定義されているので、一つの型はいくつもの型クラスのインスタンスになることができるし、
一つの型クラスはいくつもの型をインスタンスに持つことができる。

例えば`Char`型は`Eq`型クラスと`Ord`型クラスのインスタンスであり、文字の等値性比較とアルファベット順比較の両方ができる。

型をある型クラスのインスタンスにするには一旦別の型クラスのインスタンスにする必要がある。
`Ord`型クラスのインスタンスにするには、先に`Eq`型クラスのインスタンスにしておく必要がある。
`Ord`型クラスが`compare`関数で`Ordering`型の`EQ`を返しうるということを考えても納得がいく。

先ほどの`Num`型の例も同じで、先に`Show`と`Eq`のインスタンスにしないといけない。